# **Memory**

The memory allows a Large Language Model (LLM) to remember previous interactions with the user. By default, LLMs are stateless — meaning each incoming query is processed independently of other interactions.

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain langchain_community langchain-openai
!pip install --upgrade python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["OPENAI_API_VERSION"] = os.getenv('OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv('AZURE_OPENAI_API_KEY')

# **ConversationBufferMemory**

ConversationBufferMemory usage is straightforward. It simply keeps the entire conversation in the buffer memory up to the allowed max limit (e.g. 4096 for gpt-3.5-turbo, 8192 for gpt-4). The main downside, however, is the cost. Each request is sending the aggregation to the API. Since the charge is based on token usages, the cost can quickly add up, especially if you are sending requests to an AI platform. Additionally, there can be added latency given the sheer size of the text that’s being passed back and forth.

In [3]:
# ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    temperature = 0.5
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose= True
)

/tmp/ipython-input-4-158765928.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipython-input-4-158765928.py:7: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [5]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello, Andrew! It's great to meet you. My name doesn't have the same ring to it as yours—I'm just an AI—but I'm here to chat, answer questions, and share all sorts of interesting information. How can I help you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. My name doesn't have the same ring to it as yours—I'm just an AI—but I'm here to chat, answer questions, and share all sorts of interesting information. How can I help you today?
Human: What is 1+1?
AI:

> Finished chain.


"Ah, the classic math question! 1 + 1 equals 2. It's one of the simplest and most fundamental equations in arithmetic, but it's also a great starting point for understanding how numbers work. Let me know if you'd like to dive into anything more complex or interesting!"

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. My name doesn't have the same ring to it as yours—I'm just an AI—but I'm here to chat, answer questions, and share all sorts of interesting information. How can I help you today?
Human: What is 1+1?
AI: Ah, the classic math question! 1 + 1 equals 2. It's one of the simplest and most fundamental equations in arithmetic, but it's also a great starting point for understanding how numbers work. Let me know if you'd like to dive into anything more complex or interesting!
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew! You just introduced yourself a moment ago. I’ve got a pretty good memory for conversations, so feel free to ask me about anything we’ve talked about so far. 😊'

In [8]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. My name doesn't have the same ring to it as yours—I'm just an AI—but I'm here to chat, answer questions, and share all sorts of interesting information. How can I help you today?
Human: What is 1+1?
AI: Ah, the classic math question! 1 + 1 equals 2. It's one of the simplest and most fundamental equations in arithmetic, but it's also a great starting point for understanding how numbers work. Let me know if you'd like to dive into anything more complex or interesting!
Human: What is my name?
AI: Your name is Andrew! You just introduced yourself a moment ago. I’ve got a pretty good memory for conversations, so feel free to ask me about anything we’ve talked about so far. 😊


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello, Andrew! It's great to meet you. My name doesn't have the same ring to it as yours—I'm just an AI—but I'm here to chat, answer questions, and share all sorts of interesting information. How can I help you today?\nHuman: What is 1+1?\nAI: Ah, the classic math question! 1 + 1 equals 2. It's one of the simplest and most fundamental equations in arithmetic, but it's also a great starting point for understanding how numbers work. Let me know if you'd like to dive into anything more complex or interesting!\nHuman: What is my name?\nAI: Your name is Andrew! You just introduced yourself a moment ago. I’ve got a pretty good memory for conversations, so feel free to ask me about anything we’ve talked about so far. 😊"}

In [10]:
memory = ConversationBufferMemory()

In [11]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [14]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

# **ConversationBufferWindowMemory**

Given that most conversations will not need context from several messages before, we also have the option of also using ConversationBufferWindowMemory. With this option we can simply control the buffer with a window of the last k messages.

In [15]:
# ConversationBufferWindowMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [16]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    temperature = 0.5
)

memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose= True
)

/tmp/ipython-input-16-1653144920.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [17]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hi Andrew! It's great to meet you. My name doesn't really have a personal touch like yours, but you can call me AI or Assistant—whatever feels right to you. How's your day going so far?"

In [18]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hi Andrew! It's great to meet you. My name doesn't really have a personal touch like yours, but you can call me AI or Assistant—whatever feels right to you. How's your day going so far?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2! It\'s one of the simplest and most fundamental equations in mathematics. Fun fact: this concept is part of what\'s called "elementary arithmetic," which forms the basis for all kinds of advanced mathematical ideas. If you ever want to dive deeper into math, let me know!'

In [19]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 equals 2! It's one of the simplest and most fundamental equations in mathematics. Fun fact: this concept is part of what's called "elementary arithmetic," which forms the basis for all kinds of advanced mathematical ideas. If you ever want to dive deeper into math, let me know!
Human: What is my name?
AI:

> Finished chain.


'I don’t know your name, since you haven’t shared it with me yet! If you’d like, you can tell me, and I’ll be happy to remember it for the rest of our conversation. What should I call you?'

In [20]:
memory.buffer

'Human: What is my name?\nAI: I don’t know your name, since you haven’t shared it with me yet! If you’d like, you can tell me, and I’ll be happy to remember it for the rest of our conversation. What should I call you?'

# **ConversationTokenBufferMemory**

ConversationTokenBufferMemory simply maintains the buffer by the token count and does not maintain. There is no summarizing with this approach . In fact, it is similar to ConversationBufferWindowMemory, except instead of flushing based on the last number of messages (k), flushing is based on the number of tokens.

In [21]:
# ConversationTokenBufferMemory
from langchain.memory import ConversationTokenBufferMemory

In [22]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-4o",  # Use deployment_name for Azure
    model_name="gpt-4o",       # Required for token counting logic

)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

/tmp/ipython-input-22-3816489103.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)


In [23]:
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"},
                    {"output": "Charming!"})
memory.save_context({"input": "ML is what?"},
                    {"output": "MachineLearning!"})

In [24]:
memory.load_memory_variables({})

{'history': 'Human: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!\nHuman: ML is what?\nAI: MachineLearning!'}

# **ConversationSummaryMemory**

ConversationSummaryMemory does not keep the entire history in memory like ConversationBufferMemory. Nor does it maintain a window. Rather, the ConversationSummaryMemory continually summarizes the conversation as it’s happening, and maintains it so we have context from the beginning of the conversation.

In [25]:
# ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

In [26]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    model_name="gpt-4o",

)

In [27]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"},
                    {"output": f"{schedule}"})

/tmp/ipython-input-27-1139844937.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [28]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and then asks the AI about the schedule for the day.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [29]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [30]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and then asks the AI about the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A great demo to showcase during your lunch meeting would depend on your audience\'s familiarity with AI and their interests. Since they\'re driving over an hour to learn about the latest in AI, they’re probably eager to see something cutting-edge yet accessible. Here are some ideas tailored for that scenario:\n\n1. **Custom Question-Answering with Documents (LangChain + OpenAI)**: \n   - If you’ve had time to fine-tune something with LangChain, consider showing a live demo of a custom AI that uses a document or dataset related to their industry. For example, if they\'re from healthcare, demonstrate the AI helping to answer queries from abstracts or research papers. This is always a hit because it shows how AI can boost productivity in specialized fields.\n\n2. **Conversational Agent with Plugins**:\n   - Showcase a chatbot that employs plugins or tools, like retrieving real-time data, summarizing recent news articles, or performing small calculations seamlessly within a conversation—j

In [31]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI, and they exchange casual pleasantries. The human mentions they are just hanging out and then asks the AI about the schedule for the day. The AI outlines the day's schedule, including an 8 a.m. meeting with the product team, time to work on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in the latest in AI. The human asks for demo suggestions for the lunch meeting, and the AI provides several tailored ideas, including custom question-answering with LangChain, a conversational agent with plugins, workflow automation, code generation or debugging support, creative applications like story writing, and multimodal demonstrations such as GPT-4 Vision analysis. The AI emphasizes the importance of relevance, simplicity, and preparation for a smooth presentation."}

# **Let's Do an Activity**

## **Objective**

Explore different memory strategies in LangChain for managing conversational context and improving interaction quality.

## **Scenario**

You are developing a conversational AI system that interacts with users on various topics, retaining context across multiple messages. Your goal is to implement and compare different memory strategies offered by LangChain to maintain conversation history and enhance user engagement.

## **Steps**

* Choose Memory Strategy

  * ConversationBufferMemory
  * ConversationBufferWindowMemory
  * ConversationTokenBufferMemory
  * ConversationSummaryMemory

* Implement Conversation Chain
* Interact with the System
* Evaluate